# LMTrain-IPU

In this notebook we show how we do authorial GPT-2s training on Graphcore IPU.

Install dependencies.

In [ ]:
!pip install -q "optimum-graphcore>=0.5, <0.6"
!pip install transformers datasets

Import packages.

In [ ]:
import os,math,shutil
from transformers import AutoTokenizer, DataCollatorForLanguageModeling, AutoModelForCausalLM, Trainer, default_data_collator
from optimum.graphcore import IPUConfig, IPUTrainer, IPUTrainingArguments
from datasets import Dataset

Set up variables here.

log_home: the directory containing two text files done.txt and target.txt. To start, place the author_tags line by line in target.txt and leave done.txt blank. After an authorial GPT-2 is successfully fine-tuned and saved to hard drive, its label will be logged in done.txt and hence be skipped when the notebook re-started.

model_home: the directory for the fine-tuned model output.

data_path: the directory for the dataset. Typically under such directory should have two csv: train.csv and test.csv

author_tag_label: the column name for author_tag, in GEFA, it is "author_tag". Change this only when you want to use this notebook for a dataset other than GEFA series.

In [ ]:
log_home="log"
model_home="model"
data_path="corpus/GEFA-full"
author_tag_label="author_tag"

Load working environment.

In [ ]:
target_log=os.path.join(log_home,"target.txt")
done_log=os.path.join(log_home,"done.txt")
train_data_path=os.path.join(data_path,"train.csv")
if(os.path.isdir("gpt2")): shutil.rmtree("gpt2")
if(not(os.path.isdir(model_home))):os.mkdir(model_home)

Set up training tasks.

In [ ]:
with open(target_log,"r",-1,"utf-8") as f:
    target_content=f.read().strip().rstrip()
with open(done_log,"r",-1,"utf-8") as f:
    done_content=f.read().strip().rstrip()
target_lines=target_content.split()
done_lines=done_content.split()

to_do_lines=[target_line for target_line in target_lines if target_line not in done_lines]

print("Fetched tasks:")
print("\n".join(to_do_lines))
print("******")

Load dataset.

In [ ]:
total_dataset=Dataset.from_csv(train_data_path)

Run the script below to start training.

In [ ]:
for to_do_line in to_do_lines:
    to_do_line_cells=to_do_line.split(",")
    author_tag=to_do_line_cells[0]
    
    print(f"Start processing:{author_tag}")

    target_train_set=total_dataset.filter(lambda daton:daton[author_tag_label]==author_tag)
    
    dataset=target_train_set.train_test_split(test_size=0.2,shuffle=True)

    tokenizer=AutoTokenizer.from_pretrained("gpt2")
    
    def preprocess_function(examples):
        return tokenizer(examples["text"])
    
    tokenizer.pad_token=tokenizer.eos_token
    
    lm_dataset=dataset.map(preprocess_function, batched=True,num_proc=8,remove_columns=dataset["train"].column_names)
    
    def group_texts(examples,block_size=128):
        concatenated_examples={key: sum(examples[key], []) for key in examples.keys()}
        total_length=len(concatenated_examples[list(examples.keys())[0]])
        if total_length >= block_size:
            total_length=(total_length // block_size) * block_size
        result={
            key:[token[i : i + block_size] for i in range(0, total_length, block_size)]
            for key, token in concatenated_examples.items()
        }
        result["labels"]=result["input_ids"].copy()
        return result
    
    lm_dataset=lm_dataset.map(group_texts,batched=True,num_proc=8)

    tokenizer.pad_token=tokenizer.eos_token
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    pod_type=os.getenv("GRAPHCORE_POD_TYPE", "pod4")
    executable_cache_dir=os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/") + "/language_modelling_from_scratch"

    model_checkpoint="gpt2"
    ipu_config_name="Graphcore/gpt2-small-ipu"
    micro_batch_size=2
    gradient_accumulation_steps=64
    dataloader_workers=52

    ipu_config=IPUConfig.from_pretrained(ipu_config_name, executable_cache_dir=executable_cache_dir)
    model=AutoModelForCausalLM.from_pretrained(model_checkpoint)

    training_args=IPUTrainingArguments(
        f"{model_checkpoint}",
        learning_rate=2e-5,
        weight_decay=0.01,
        per_device_train_batch_size=micro_batch_size,
        per_device_eval_batch_size=micro_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        num_train_epochs=100,
        loss_scaling=16384,
        pod_type=pod_type,
        warmup_ratio=0.1,
        dataloader_drop_last=True,
        dataloader_num_workers=dataloader_workers,
        logging_steps=10,
        push_to_hub=False,
        save_total_limit=2,
    )
    trainer=IPUTrainer(
        model=model,
        ipu_config=ipu_config,
        args=training_args,
        train_dataset=lm_dataset["train"],
        eval_dataset=lm_dataset["test"],
        tokenizer=tokenizer,
        data_collator=default_data_collator,
    )
    trainer.train()
    trainer.save_model(os.path.join(model_home,author_tag))

    eval_results=trainer.evaluate()
    eval_str=f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}"
    print(eval_str)
    
    with open(os.path.join(model_home,author_tag,"eval.txt"),"a",-1,"utf-8") as f:
        f.write(eval_str)
    
    with open(done_log,"a",-1,"utf-8") as f:
        f.write(to_do_line+"\n")
        
    if(os.path.isdir("gpt2")): shutil.rmtree("gpt2")
        
    print(f"Finished processing:{author_tag}")